In [1]:
import pandas as pd
import numpy as np

import datetime

from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def fillnan(data, years_c):
    for name in years_c:
        #заменяем троеточия
        data[name] = data[name].apply(lambda x: np.nan if x==('..') else float(x))

    return data

def dropempt(data, years_c):
    data = fillnan(data, years_c)
    return data.dropna(thresh=10)

def Clearing(data):
    years_c = [item for ind, item in enumerate(np.array(data.columns)) if item not in ['Country Name', 'Country Code','Series Code','Series Name']]
    print ('Before clear:', len(data))

    data = dropempt(data, years_c)

    print ('After clear:', len(data))
    return data

In [3]:
def Make_region(code, reg='region', by = 'name'):
    #print code
    if code in data_cnt[by].values:
        #print code
        name = (data_cnt[by][data_cnt[by] == code].index.tolist())[0]
        return data_cnt[reg].at[name]
    else:
        return np.nan

def Sorting(data, reg='region'):
    #if data['Country Name'].get_value(len(data)-5) == np.nan:
    #    data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    if 'Region' not in data.columns:
        data['Region'] = data['Country Name'].progress_apply(lambda x: Make_region(x, reg))
        data['Region'].dropna()
        return data.reset_index()
    else:
        return data

In [4]:
df = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', encoding = 'utf8').\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics_by_Wealth_Quintile.xlsx',encoding='utf8'))

data_cnt = pd.read_csv('all.csv', encoding='utf8')
df = Clearing(df)
defen = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', sheet_name=1, encoding = 'utf8').\
               append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheet_name=1, encoding='utf8')).\
               append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', sheet_name=1, encoding='utf8'))

Before clear: 398541
After clear: 104445


In [5]:
df = Sorting(df, 'sub-region')

HBox(children=(IntProgress(value=0, max=104445), HTML(value='')))

In [6]:
df.drop_duplicates(inplace = True)

In [7]:
years_c = [item for ind, item in enumerate(np.array(df.columns)) if item not in ['Region','Country Name', 'Country Code','Series Code','Series Name', 'index']]

In [8]:
df[(df['Series Name'] != 'Life expectancy at birth, total (years)')].head()

,index,2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],Country Code,Country Name,Series Code,Series Name,Region
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,3.000000,3.000000,5.000000,5.000000,10.000000,AFG,Afghanistan,SH.HIV.ARTC.FE.ZS,"Access to anti-retroviral drugs, female (%)",Southern Asia
1,2,0.000000,0.000000,0.000000,6.000000,8.000000,9.000000,17.000000,26.000000,29.000000,34.000000,45.000000,57.000000,67.000000,78.000000,DZA,Algeria,SH.HIV.ARTC.FE.ZS,"Access to anti-retroviral drugs, female (%)",Northern Africa
2,5,0.000000,0.000000,0.000000,3.000000,5.000000,7.000000,9.000000,11.000000,13.000000,14.000000,20.000000,24.000000,26.000000,24.000000,AGO,Angola,SH.HIV.ARTC.FE.ZS,"Access to anti-retroviral drugs, female (%)",Middle Africa
3,7,1.957837,2.349114,2.886862,4.131851,5.251858,7.379121,8.981335,10.809919,12.393742,14.735615,17.476939,20.271829,23.921017,27.819662,ARB,Arab World,SH.HIV.ARTC.FE.ZS,"Access to anti-retroviral drugs, female (%)",NaN
4,8,40.000000,44.000000,48.000000,51.000000,55.000000,58.000000,60.000000,63.000000,65.000000,68.000000,70.000000,72.000000,74.000000,75.000000,ARG,Argentina,SH.HIV.ARTC.FE.ZS,"Access to anti-retroviral drugs, female (%)",South America


In [9]:
df.drop(columns = 'index', inplace=True)

Life expectancy at birth, total (years)

In [31]:
def makexy(df, country = 'RUS'):
    #Years
    years_c = [item for ind, item in enumerate(np.array(df.columns)) if item not in ['Region','Country Name', 'Country Code','Series Code','Series Name', 'index']]
    
    #split country
    df = df[(df['Country Code']== country)].copy()
    df.drop_duplicates(inplace = True)
    
    y = df[df['Series Name'] == 'Life expectancy at birth, total (years)'][years_c].interpolate(axis=1).T.as_matrix()
    #Transponenting and then adding 
    X = df[df['Series Name'] != 'Life expectancy at birth, total (years)'][years_c].T
    X.columns = df[(df['Country Code']== country) & (df['Series Name'] != 'Life expectancy at birth, total (years)')]['Series Name'].tolist()
    X.reset_index(inplace = True, drop = True)
    del df
    
    X['Years'] = [int(name.split()[0]) for name in years_c]
    X['Region_code'] = int(Make_region(country,'region-code', by = 'alpha-3'))
    X['Reion_sub_code'] = int(Make_region(country, 'sub-region-code', 'alpha-3'))
    X['Country_code'] = int(Make_region(country, 'country-code', 'alpha-3'))
    return X.as_matrix(), y, X.columns

In [32]:
def Testing_grid_xgb(X_train, Y_train):
    cv = TimeSeriesSplit(n_splits=5)
    clf = XGBRegressor()        
    parameters = {
        'scale_pos_weight' : [1.5],
        'objective':['reg:linear'],
        'learning_rate': [0.3, 0.4],
        'max_depth': [6],
        'min_child_weight': [4],
        'max_delta_step': [0],
        'reg_lambda': [0],
        'reg_alpha': [0.8],
        'silent': [1],
        'subsample': [0.05],
        'colsample_bytree': [1],
        'n_estimators': [150, 450],
        'base_score' : [0.5]
    }
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='neg_mean_absolute_error', param_grid=parameters, cv=cv,  n_jobs = -1, verbose=True)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    
    print (gs.best_params_)
    return max(means), gs.best_params_

In [33]:
tmp = makexy(df)

In [34]:
Testing_grid_xgb(tmp[0], tmp[1])

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Time elapsed: 0:00:00.495474
{'base_score': 0.5, 'colsample_bytree': 1, 'learning_rate': 0.4, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 450, 'objective': 'reg:linear', 'reg_alpha': 0.8, 'reg_lambda': 0, 'scale_pos_weight': 1.5, 'silent': 1, 'subsample': 0.05}


[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished


(-1.2826133675924196,
 {'base_score': 0.5,
  'colsample_bytree': 1,
  'learning_rate': 0.4,
  'max_delta_step': 0,
  'max_depth': 6,
  'min_child_weight': 4,
  'n_estimators': 450,
  'objective': 'reg:linear',
  'reg_alpha': 0.8,
  'reg_lambda': 0,
  'scale_pos_weight': 1.5,
  'silent': 1,
  'subsample': 0.05})

In [35]:
model = XGBRegressor(max_depth=6, learning_rate=0.3, n_estimators=450, 
                     silent=True, objective='reg:linear', booster='gbtree', 
                     nthread=None, gamma=0, min_child_weight=4,
                     max_delta_step=0, subsample=0.05, colsample_bytree=1, 
                     colsample_bylevel=1, reg_alpha=0.8, reg_lambda=1, 
                     scale_pos_weight=1.5, base_score=0.5, random_state=41).fit(tmp[0], tmp[1])

In [48]:
pd.DataFrame(model.feature_importances_, index = tmp[2]).sort_values(by = [0], ascending=False).head()

,0
"Adjusted net enrollment rate, primary, female (% of primary school age children)",0.923077
"Adolescent fertility rate (births per 1,000 women ages 15-19)",0.076923
"Mortality rate, adult, male (per 1,000 male adults)",0.000000
People using basic drinking water services (% of population),0.000000
Out-of-pocket health expenditure (% of total expenditure on health),0.000000
